In [1]:
from transformers import T5Tokenizer, T5ForConditionalGeneration, AutoModel, AutoTokenizer
from experimental.t5_dataset import T5Dataset
import json
import os
import os.path as op

st_tokenizer = AutoTokenizer.from_pretrained("sentence-transformers/msmarco-bert-base-dot-v5") #model_args={"embedding_size":512})
st_model = AutoModel.from_pretrained("sentence-transformers/msmarco-bert-base-dot-v5")#embedding_size=512)


t5_tokenizer = T5Tokenizer.from_pretrained("t5-base", extra_ids=1)


########Building Frankenstein's monster #########
t5_model = T5ForConditionalGeneration.from_pretrained("t5-base")

from experimental.test_t5 import STOnlyEncoder, CustomEncoder
t5_model.encoder = CustomEncoder(t5_model.encoder)
#t5_model.encoder = STOnlyEncoder(st_model, st_tokenizer=st_tokenizer, t5_tokenizer=t5_tokenizer) #Unfortunately all these parameters are necessary

######### Monster building complete ##########

#ORIGINAL_t5_model = T5ForConditionalGeneration.from_pretrained("t5-base")

DATA_DIR = "DialogSum_Data"

SAMPLE_DATA = op.join(DATA_DIR, 'dialogsum.sample.jsonl')
TEST_DATA = op.join(DATA_DIR, 'dialogsum.test.jsonl')
TRAIN_DATA = op.join(DATA_DIR, 'dialogsum.train.jsonl')
DEV_DATA = op.join(DATA_DIR, 'dialogsum.dev.jsonl')

def load_jsonl(filepath):
    
    output_list = []
    
    with open(filepath) as sd_file:
        lines = sd_file.readlines()
    
    for line in lines:
        output_list.append(json.loads(line))

    return output_list


if __name__ == "__main__":
    dev_data_list = load_jsonl(DEV_DATA)
    dev_dataset = T5Dataset(dev_data_list, 
                  tokenizer=t5_tokenizer,
                  debug=True)   
    for i in dev_dataset:
        print(i)


{'source_ids': tensor([ 1713,   345, 13515,   536,  4663,    10,  8774,     6,   149,    33,
           25,   692,   469, 32000,  5780,   357,  4663,    10,    27,     3,
           31,  3901,   118,   578,  3169, 10882, 12643, 32000,  5780,   536,
         4663,    10,  2114,    25,   141,   136,   686,    13,  2107, 12643,
        32000,  5780,   357,  4663,    10,   465,     6,    27,    43,    29,
            3,    31,     3,    17,   141,     3,     9,  2107,     5,    27,
          131,    43,     3,     9,  2437,  1829,    16,    82,  5738,   116,
           27,   653,    12, 13418, 32000,  5780,   536,  4663,    10,   531,
           25,    43,   136, 18500,    24,    25,   214,    13, 32000,  5780,
          357,  4663,    10,   465,     6,    27,   278,     3,    31,     3,
           17,    43,   136, 18500,    24,    27,   214,    13, 32000,  5780,
          536,  4663,    10,  3520,    48,  1837,    66,     8,    97,    42,
         3323,   116,    25,    33,  1676, 32000,

Token indices sequence length is longer than the specified maximum sequence length for this model (584 > 512). Running this sequence through the model will result in indexing errors


{'source_ids': tensor([ 1713,   345, 13515,   536,  4663,    10, 20335,  1074,   140,     5,
           27,    31,    51,     3,     9, 13037,   270,    11,  1513,    82,
          194, 32000,  5780,   357,  4663,    10,  1072,    27,   199,    25,
        32000,  5780,   536,  4663,    10, 10625,     5,  1072,    27,   129,
           12,     8,  2069,  3066,     3,  9176,    48,   194, 32000,  5780,
          357,  4663,    10,     3,    76,   107,   233,  2163,     5,  5527,
          269,     6,   258,   240,     8,   511,  5074,    30,    39,   646,
        32000,  5780,   536,  4663,    10,    27,     7,    34,   623, 32000,
         5780,   357,  4663,    10,    94,    31,     7,    81, 17310,   676,
           31,  1482,     5,   466,    31,     7,    66, 32000,  5780,   536,
         4663,    10,   275,   103,    25,   214,   213,     8,  1157,  2137,
           19, 32000,  5780,   357,  4663,    10,  2163,     5,    94,    31,
            7,    30,    48,  2815,     5,  3521,

In [2]:
from IPython.display import clear_output

dev_dataset = T5Dataset(dev_data_list, 
                tokenizer=t5_tokenizer,
                debug=True)   

for i in dev_dataset:
    ids = i["source_ids"]
    mask = i["source_mask"]
   
    # print("#"*20)
    # print(ids)
    # print(mask)
    # print(dir(mask))
    # print(dir(t5_model))
   # print(type(ids))
   # print(type(mask))
    ORIG_encoding = t5_tokenizer.batch_encode_plus("some sentence here", padding=True, return_tensors="pt")
    ORIG_ids = ORIG_encoding["input_ids"]
    ORIG_mask = ORIG_encoding["attention_mask"]
    output = t5_model.generate(input_ids = ORIG_ids, attention_mask = ORIG_mask)
    # print(ORIG_output)
    # print("o"*10)
    # print(ORIG_ids.shape)
    print(ids.shape)
    # print("f"*10)
    # print(ORIG_mask.shape)
    print(mask.shape)

    # print()
    # print(ORIG_encoding)
    # break
    
    output = t5_model.generate(input_ids=ids, attention_mask = mask)
    print(t5_tokenizer.decode(output))
    break

BaseModelOutputWithPastAndCrossAttentions(last_hidden_state=tensor([[[-1.6156e-01, -2.8336e-01, -1.2733e-01,  ...,  1.1558e-01,
          -1.5975e-01, -1.2519e-01],
         [-4.7700e-01, -1.0188e-01, -1.0653e-01,  ..., -2.6955e-01,
           2.6203e-01, -3.9815e-01],
         [ 5.9081e-03, -1.3118e-02, -2.0144e-03,  ..., -2.8925e-02,
           1.0382e-02, -1.3435e-03]],

        [[-8.3076e-02, -2.8301e-01, -5.1458e-02,  ...,  1.7123e-01,
           1.9964e-01, -2.1760e-01],
         [-3.0605e-01,  5.5673e-02, -2.2927e-01,  ...,  6.2846e-02,
          -1.1734e-01, -5.6387e-01],
         [ 4.5536e-03,  5.8000e-03, -9.5827e-03,  ..., -3.7586e-02,
          -5.5778e-03, -5.3566e-03]],

        [[-6.7033e-02, -3.2600e-01, -1.9768e-01,  ...,  1.1949e-01,
          -2.3814e-01,  8.2863e-02],
         [-2.9059e-01, -3.2293e-02, -3.2694e-02,  ..., -1.5450e-01,
          -2.1481e-01, -5.2311e-01],
         [-5.3758e-03, -1.8778e-04,  5.2100e-04,  ..., -2.3595e-02,
           2.4085e-02, -6.41

NameError: name 'exit' is not defined